# What is Transfer Learning?

Transfer learning, is a research problem in machine learning that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem

# Why Transfer Learning?

- In practice a very few people train a Convolution network from scratch (random initialisation) because it is rare to get enough dataset. So, using pre-trained network weights as initialisations or a fixed feature extractor helps in solving most of the problems in hand.
- Very Deep Networks are expensive to train. The most complex models take weeks to train using hundreds of machines equipped with expensive GPUs.
- Determining the topology/flavour/training method/hyper parameters for deep learning is a black art with not much theory to guide you.

# How Transfer Learning helps ?


When you look at what these Deep Learning networks learn, they try to detect edges in the earlier layers, Shapes in the middle layer and some high level data specific features in the later layers. These trained networks are generally helpful in solving other computer vision problems. Lets have a look at how to do transfer learning using Keras and various cases in Transfer learning.

In [0]:
from keras.applications import vgg19
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten
from keras.datasets import cifar10
from keras.utils import np_utils

In [0]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
xtrain = xtrain.astype('float32')          #float conversion
xtest = xtest.astype('float32')
xtrain = xtrain/255         #specific preprocessing applied 
xtest = xtest/255
ytrain = np_utils.to_categorical(ytrain, 10)  #conversion of output into matrix
ytest = np_utils.to_categorical(ytest, 10)

In [0]:
# transfer learning model
model=vgg19.VGG19(weights="imagenet",include_top=False,input_shape=(32,32,3))

# adding custom layers using Functional model api of keras
x= model.output
x= Flatten()(x)
x= Dense(256,activation='relu')(x)
pred=Dense(10,activation="softmax")(x)

#creating the final model
model=Model(inputs=model.input,outputs=pred)

# setting no. of untrained layers in our final model
for layer in model.layers[:-3]:
    layer.trainable = False
    
model.compile(loss="categorical_crossentropy",
              optimizer="Adadelta",
              metrics=['accuracy'])

In [16]:
model.fit(xtrain, ytrain,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(xtest, ytest))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 11s 216us/step - loss: 1.4891 - acc: 0.4754 - val_loss: 1.4244 - val_acc: 0.5004
Epoch 2/20
50000/50000 [==============================] - 10s 207us/step - loss: 1.2870 - acc: 0.5477 - val_loss: 1.2579 - val_acc: 0.5616
Epoch 3/20
50000/50000 [==============================] - 10s 207us/step - loss: 1.2208 - acc: 0.5734 - val_loss: 1.2403 - val_acc: 0.5624
Epoch 4/20
50000/50000 [==============================] - 11s 211us/step - loss: 1.1811 - acc: 0.5830 - val_loss: 1.2395 - val_acc: 0.5689
Epoch 5/20
50000/50000 [==============================] - 11s 213us/step - loss: 1.1445 - acc: 0.5968 - val_loss: 1.2155 - val_acc: 0.5775
Epoch 6/20
50000/50000 [==============================] - 11s 213us/step - loss: 1.1176 - acc: 0.6059 - val_loss: 1.2196 - val_acc: 0.5723
Epoch 7/20
50000/50000 [==============================] - 11s 212us/step - loss: 1.0900 - acc: 0.6164 - val_loss: 1.

In [17]:
score = model.evaluate(xtest, ytest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.1508881008148193
Test accuracy: 0.6064
